# 53-异步性能优化

## 📚 用途说明

**学习目标**：
- 掌握异步性能分析和监控工具
- 学会事件循环优化和任务调度策略
- 理解内存管理和泄漏检测技术
- 能够进行异步基准测试和性能对比

**前置要求**：
- 已完成异步任务队列学习
- 熟练掌握异步编程核心概念
- 了解基本的性能测试方法
- 理解并发编程模式和实践

**与LangChain关联**：
- 性能优化提升LangChain的响应速度
- 监控工具确保LangChain的稳定性
- 基准测试验证LangChain的优化效果
- 内存管理保障LangChain的大规模部署

---

## 🔢 知识点覆盖

### 6.8 异步性能优化 [⭐⭐进阶]
**知识点说明**：异步性能优化是构建高性能异步系统的关键技能，对于LangChain的大规模应用和企业级部署具有重要意义。

**学习要求**：
- 掌握异步性能分析和监控工具
- 学会事件循环优化和任务调度策略
- 理解内存管理和泄漏检测技术
- 能够进行异步基准测试和性能对比

**案例要求**：
- 实现完整的性能分析工具集
- 进行异步系统性能优化实践
- 应用优化技术解决实际问题
- 验证点：能独立进行异步系统性能调优

In [ ]:
print("📊 异步性能优化:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import json
import random
import uuid
import gc
import tracemalloc
import resource
import threading
import psutil
import hashlib
from typing import List, Dict, Any, Optional, Callable, Union
from dataclasses import dataclass, asdict, field
from enum import Enum
from collections import defaultdict, deque
import weakref
import inspect
import functools

# 性能分析相关
import cProfile
import pstats
import io
from contextlib import contextmanager, asynccontextmanager

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 异步性能分析工具
print(f"📝 1. 异步性能分析工具:")

# 1.1 性能监控器
print(f"\n   🔍 1.1 性能监控器:")

@dataclass
class PerformanceMetrics:
    """性能指标"""
    timestamp: float = field(default_factory=time.time)
    cpu_percent: float = 0.0
    memory_mb: float = 0.0
    memory_percent: float = 0.0
    active_tasks: int = 0
    completed_tasks: int = 0
    failed_tasks: int = 0
    average_response_time: float = 0.0
    throughput: float = 0.0  # 任务/秒
    event_loop_lag: float = 0.0

class AsyncPerformanceMonitor:
    """异步性能监控器"""
    
    def __init__(self, sampling_interval: float = 1.0):
        self.sampling_interval = sampling_interval
        self.metrics_history: List[PerformanceMetrics] = []
        self.task_metrics: Dict[str, Dict[str, float]] = {}
        self.start_time = time.time()
        self.monitoring = False
        self.monitor_task: Optional[asyncio.Task] = None
        
        # 统计计数器
        self.task_counter = {
            "total": 0,
            "completed": 0,
            "failed": 0,
            "active": 0
        }
        
        # 响应时间统计
        self.response_times: deque = deque(maxlen=1000)
        
        # 进程对象
        self.process = psutil.Process()
    
    async def start_monitoring(self):
        """开始监控"""
        if self.monitoring:
            return
        
        self.monitoring = True
        self.start_time = time.time()
        
        # 启动监控任务
        self.monitor_task = asyncio.create_task(self._monitor_loop())
        
        print(f"      🚀 性能监控启动 (采样间隔: {self.sampling_interval}s)")
    
    async def stop_monitoring(self):
        """停止监控"""
        if not self.monitoring:
            return
        
        self.monitoring = False
        
        if self.monitor_task:
            self.monitor_task.cancel()
            try:
                await self.monitor_task
            except asyncio.CancelledError:
                pass
        
        print(f"      🛑 性能监控停止")
    
    def record_task_start(self, task_id: str):
        """记录任务开始"""
        self.task_counter["total"] += 1
        self.task_counter["active"] += 1
        
        self.task_metrics[task_id] = {
            "start_time": time.time()
        }
    
    def record_task_complete(self, task_id: str, success: bool = True):
        """记录任务完成"""
        if task_id in self.task_metrics:
            end_time = time.time()
            start_time = self.task_metrics[task_id]["start_time"]
            response_time = end_time - start_time
            
            self.task_metrics[task_id].update({
                "end_time": end_time,
                "response_time": response_time,
                "success": success
            })
            
            self.response_times.append(response_time)
            
            self.task_counter["active"] -= 1
            
            if success:
                self.task_counter["completed"] += 1
            else:
                self.task_counter["failed"] += 1
    
    async def _monitor_loop(self):
        """监控循环"""
        while self.monitoring:
            try:
                metrics = await self._collect_metrics()
                self.metrics_history.append(metrics)
                
                # 限制历史记录长度
                if len(self.metrics_history) > 1000:
                    self.metrics_history = self.metrics_history[-1000:]
                
                await asyncio.sleep(self.sampling_interval)
                
            except Exception as e:
                print(f"      ❌ 监控循环错误: {e}")
                await asyncio.sleep(1)
    
    async def _collect_metrics(self) -> PerformanceMetrics:
        """收集性能指标"""
        # CPU和内存使用情况
        cpu_percent = self.process.cpu_percent()
        memory_info = self.process.memory_info()
        memory_mb = memory_info.rss / 1024 / 1024
        memory_percent = self.process.memory_percent()
        
        # 事件循环延迟
        loop = asyncio.get_event_loop()
        event_loop_lag = loop.time() - time.time()
        
        # 平均响应时间
        if self.response_times:
            average_response_time = sum(self.response_times) / len(self.response_times)
        else:
            average_response_time = 0.0
        
        # 吞吐量计算
        elapsed_time = time.time() - self.start_time
        if elapsed_time > 0:
            throughput = self.task_counter["completed"] / elapsed_time
        else:
            throughput = 0.0
        
        return PerformanceMetrics(
            cpu_percent=cpu_percent,
            memory_mb=memory_mb,
            memory_percent=memory_percent,
            active_tasks=self.task_counter["active"],
            completed_tasks=self.task_counter["completed"],
            failed_tasks=self.task_counter["failed"],
            average_response_time=average_response_time,
            throughput=throughput,
            event_loop_lag=event_loop_lag
        )
    
    def get_current_metrics(self) -> Optional[PerformanceMetrics]:
        """获取当前指标"""
        return self.metrics_history[-1] if self.metrics_history else None
    
    def get_metrics_summary(self) -> Dict[str, Any]:
        """获取指标摘要"""
        if not self.metrics_history:
            return {}
        
        # 计算统计值
        cpu_values = [m.cpu_percent for m in self.metrics_history]
        memory_values = [m.memory_mb for m in self.metrics_history]
        response_time_values = [m.average_response_time for m in self.metrics_history]
        throughput_values = [m.throughput for m in self.metrics_history]
        
        return {
            "monitoring_duration": time.time() - self.start_time,
            "total_samples": len(self.metrics_history),
            "task_summary": self.task_counter.copy(),
            "cpu": {
                "current": cpu_values[-1] if cpu_values else 0,
                "average": sum(cpu_values) / len(cpu_values) if cpu_values else 0,
                "max": max(cpu_values) if cpu_values else 0,
                "min": min(cpu_values) if cpu_values else 0
            },
            "memory": {
                "current_mb": memory_values[-1] if memory_values else 0,
                "average_mb": sum(memory_values) / len(memory_values) if memory_values else 0,
                "max_mb": max(memory_values) if memory_values else 0,
                "min_mb": min(memory_values) if memory_values else 0
            },
            "performance": {
                "average_response_time": sum(response_time_values) / len(response_time_values) if response_time_values else 0,
                "max_response_time": max(response_time_values) if response_time_values else 0,
                "average_throughput": sum(throughput_values) / len(throughput_values) if throughput_values else 0,
                "max_throughput": max(throughput_values) if throughput_values else 0
            }
        }

# 1.2 异步代码分析器
print(f"\n   🔬 1.2 异步代码分析器:")

@dataclass
class CodeAnalysisResult:
    """代码分析结果"""
    function_name: str
    execution_time: float
    memory_usage: float
    await_count: int
    blocking_calls: List[str]
    recommendations: List[str]

class AsyncCodeAnalyzer:
    """异步代码分析器"""
    
    def __init__(self):
        self.analysis_results: List[CodeAnalysisResult] = []
        self.blocking_patterns = [
            'time.sleep',
            'requests.get',
            'requests.post',
            'urllib.request',
            'socket.socket',
            'subprocess.run',
            'os.system',
            'open(',
            'file.read',
            'file.write'
        ]
    
    @contextmanager
    def analyze_function(self, func_name: str):
        """分析函数性能的上下文管理器"""
        # 开始内存跟踪
        tracemalloc.start()
        start_time = time.perf_counter()
        start_memory = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
        
        try:
            yield self
        finally:
            # 结束分析
            end_time = time.perf_counter()
            end_memory = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
            
            execution_time = end_time - start_time
            memory_usage = (end_memory - start_memory) / 1024  # KB
            
            # 获取内存快照
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            
            # 分析代码
            source = inspect.getsource(func_name) if callable(func_name) else ""
            await_count = source.count('await ')
            
            # 检查阻塞调用
            blocking_calls = []
            for pattern in self.blocking_patterns:
                if pattern in source:
                    blocking_calls.append(pattern)
            
            # 生成建议
            recommendations = self._generate_recommendations(
                execution_time, memory_usage, await_count, blocking_calls
            )
            
            result = CodeAnalysisResult(
                function_name=str(func_name),
                execution_time=execution_time,
                memory_usage=memory_usage,
                await_count=await_count,
                blocking_calls=blocking_calls,
                recommendations=recommendations
            )
            
            self.analysis_results.append(result)
    
    def _generate_recommendations(self, execution_time: float, 
                                 memory_usage: float, await_count: int,
                                 blocking_calls: List[str]) -> List[str]:
        """生成优化建议"""
        recommendations = []
        
        # 执行时间建议
        if execution_time > 1.0:
            recommendations.append("⚠️ 执行时间较长，考虑分解为更小的异步任务")
        
        # 内存使用建议
        if memory_usage > 100 * 1024:  # 100MB
            recommendations.append("⚠️ 内存使用较高，考虑使用生成器或流式处理")
        
        # 异步操作建议
        if await_count == 0:
            recommendations.append("💡 未使用异步操作，考虑添加await调用提高并发性")
        elif await_count > 10:
            recommendations.append("💡 异步操作较多，考虑使用asyncio.gather批量执行")
        
        # 阻塞调用建议
        if blocking_calls:
            recommendations.append(f"⚠️ 发现阻塞调用: {', '.join(blocking_calls)}，建议使用异步替代方案")
        
        return recommendations
    
    def get_analysis_summary(self) -> Dict[str, Any]:
        """获取分析摘要"""
        if not self.analysis_results:
            return {}
        
        total_time = sum(r.execution_time for r in self.analysis_results)
        total_memory = sum(r.memory_usage for r in self.analysis_results)
        total_awaits = sum(r.await_count for r in self.analysis_results)
        all_blocking = set()
        all_recommendations = set()
        
        for result in self.analysis_results:
            all_blocking.update(result.blocking_calls)
            all_recommendations.update(result.recommendations)
        
        return {
            "analyzed_functions": len(self.analysis_results),
            "total_execution_time": total_time,
            "average_execution_time": total_time / len(self.analysis_results),
            "total_memory_usage": total_memory,
            "average_memory_usage": total_memory / len(self.analysis_results),
            "total_await_calls": total_awaits,
            "blocking_calls_found": list(all_blocking),
            "recommendations": list(all_recommendations),
            "detailed_results": [asdict(r) for r in self.analysis_results]
        }

# 1.3 内存泄漏检测器 (新增)
print(f"\n   🔍 1.3 内存泄漏检测器:")

@dataclass
class MemorySnapshot:
    """内存快照"""
    timestamp: float = field(default_factory=time.time)
    current_memory: int = 0
    peak_memory: int = 0
    object_count: int = 0
    gc_stats: Dict[str, int] = field(default_factory=dict)

class AsyncMemoryLeakDetector:
    """异步内存泄漏检测器"""
    
    def __init__(self, snapshot_interval: float = 10.0):
        self.snapshot_interval = snapshot_interval
        self.snapshots: List[MemorySnapshot] = []
        self.object_tracker: Dict[str, weakref.ref] = {}
        self.running = False
        self.monitor_task: Optional[asyncio.Task] = None
        
        # 泄漏检测阈值
        self.memory_growth_threshold = 50 * 1024 * 1024  # 50MB
        self.object_growth_threshold = 1000  # 1000个对象
    
    async def start_monitoring(self):
        """开始内存监控"""
        if self.running:
            return
        
        self.running = True
        tracemalloc.start()
        
        # 启动监控任务
        self.monitor_task = asyncio.create_task(self._monitor_loop())
        
        print(f"      🚀 内存泄漏检测器启动 (采样间隔: {self.snapshot_interval}s)")
    
    async def stop_monitoring(self):
        """停止内存监控"""
        if not self.running:
            return
        
        self.running = False
        
        if self.monitor_task:
            self.monitor_task.cancel()
            try:
                await self.monitor_task
            except asyncio.CancelledError:
                pass
        
        tracemalloc.stop()
        print(f"      🛑 内存泄漏检测器停止")
    
    def track_object(self, obj: Any, name: str = None):
        """跟踪对象"""
        obj_id = str(id(obj))
        if name:
            obj_id = f"{name}_{obj_id}"
        
        self.object_tracker[obj_id] = weakref.ref(obj, self._object_callback)
    
    def _object_callback(self, ref):
        """对象回收回调"""
        # 查找并删除被回收的对象
        for obj_id, obj_ref in list(self.object_tracker.items()):
            if obj_ref is ref:
                del self.object_tracker[obj_id]
                break
    
    async def _monitor_loop(self):
        """监控循环"""
        while self.running:
            try:
                snapshot = await self._take_snapshot()
                self.snapshots.append(snapshot)
                
                # 检测内存泄漏
                await self._detect_memory_leaks()
                
                # 限制快照数量
                if len(self.snapshots) > 100:
                    self.snapshots = self.snapshots[-100:]
                
                await asyncio.sleep(self.snapshot_interval)
                
            except Exception as e:
                print(f"      ❌ 内存监控循环错误: {e}")
                await asyncio.sleep(5)
    
    async def _take_snapshot(self) -> MemorySnapshot:
        """获取内存快照"""
        # 获取内存使用情况
        current, peak = tracemalloc.get_traced_memory()
        
        # 获取对象计数
        object_count = len(self.object_tracker)
        
        # 获取GC统计
        gc.collect()
        gc_stats = {
            "counts": gc.get_count(),
            "thresholds": gc.get_threshold(),
            "garbage_objects": len(gc.garbage)
        }
        
        return MemorySnapshot(
            current_memory=current,
            peak_memory=peak,
            object_count=object_count,
            gc_stats=gc_stats
        )
    
    async def _detect_memory_leaks(self):
        """检测内存泄漏"""
        if len(self.snapshots) < 2:
            return
        
        # 比较最近的快照
        current = self.snapshots[-1]
        previous = self.snapshots[-2]
        
        # 检查内存增长
        memory_growth = current.current_memory - previous.current_memory
        if memory_growth > self.memory_growth_threshold:
            print(f"      ⚠️ 检测到内存增长: {memory_growth / 1024 / 1024:.1f}MB")
        
        # 检查对象增长
        object_growth = current.object_count - previous.object_count
        if object_growth > self.object_growth_threshold:
            print(f"      ⚠️ 检测到对象增长: {object_growth} 个对象")
        
        # 检查垃圾对象
        if current.gc_stats["garbage_objects"] > 0:
            print(f"      ⚠️ 检测到垃圾对象: {current.gc_stats['garbage_objects']} 个")
    
    def get_memory_analysis(self) -> Dict[str, Any]:
        """获取内存分析结果"""
        if not self.snapshots:
            return {}
        
        # 计算内存趋势
        memory_values = [s.current_memory for s in self.snapshots]
        object_counts = [s.object_count for s in self.snapshots]
        
        memory_trend = "stable"
        if len(memory_values) >= 3:
            recent_avg = sum(memory_values[-3:]) / 3
            early_avg = sum(memory_values[:3]) / 3
            if recent_avg > early_avg * 1.2:
                memory_trend = "increasing"
            elif recent_avg < early_avg * 0.8:
                memory_trend = "decreasing"
        
        return {
            "total_snapshots": len(self.snapshots),
            "current_memory_mb": memory_values[-1] / 1024 / 1024 if memory_values else 0,
            "peak_memory_mb": max(memory_values) / 1024 / 1024 if memory_values else 0,
            "tracked_objects": len(self.object_tracker),
            "memory_trend": memory_trend,
            "latest_gc_stats": self.snapshots[-1].gc_stats if self.snapshots else {}
        }

# 1.4 asyncio调试模式 (新增)
print(f"\n   🐛 1.4 asyncio调试模式:")

class AsyncDebugProfiler:
    """asyncio调试性能分析器"""
    
    def __init__(self):
        self.original_debug = None
        self.profiler = None
        self.stats_buffer = io.StringIO()
        
    def enable_debug_mode(self):
        """启用asyncio调试模式"""
        loop = asyncio.get_event_loop()
        self.original_debug = loop.get_debug()
        loop.set_debug(True)
        
        print(f"      🐛 asyncio调试模式已启用")
    
    def disable_debug_mode(self):
        """禁用asyncio调试模式"""
        loop = asyncio.get_event_loop()
        if self.original_debug is not None:
            loop.set_debug(self.original_debug)
        
        print(f"      🐛 asyncio调试模式已禁用")
    
    def start_profiling(self):
        """开始性能分析"""
        self.profiler = cProfile.Profile()
        self.profiler.enable()
        
        print(f"      📈 性能分析已开始")
    
    def stop_profiling(self) -> Dict[str, Any]:
        """停止性能分析并获取结果"""
        if not self.profiler:
            return {}
        
        self.profiler.disable()
        
        # 分析结果
        stats = pstats.Stats(self.profiler, stream=self.stats_buffer)
        stats.sort_stats('cumulative')
        stats.print_stats(20)  # 显示前20个函数
        
        # 获取统计信息
        result = {
            "profiler_output": self.stats_buffer.getvalue(),
            "total_calls": stats.total_calls,
            "primitive_calls": stats.prim_calls,
            "total_time": stats.total_tt
        }
        
        # 清空缓冲区
        self.stats_buffer.seek(0)
        self.stats_buffer.truncate(0)
        
        print(f"      📊 性能分析完成，总调用: {result['total_calls']}")
        
        return result
    
    @asynccontextmanager
    async def profile_async_context(self, description: str = "异步代码块"):
        """异步上下文性能分析"""
        print(f"      🔍 开始分析: {description}")
        self.start_profiling()
        
        try:
            yield self
        finally:
            results = self.stop_profiling()
            print(f"      ✅ 分析完成: {description}")
            return results

print(f"   ✅ 异步性能分析工具完成")
print(f"      - AsyncPerformanceMonitor: 实时性能监控")
print(f"      - AsyncCodeAnalyzer: 代码性能分析")
print(f"      - AsyncMemoryLeakDetector: 内存泄漏检测")
print(f"      - AsyncDebugProfiler: asyncio调试模式")

print(f"\n✅ 异步性能分析工具完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步性能分析和监控工具")
print(f"   ✓ 学会事件循环优化和任务调度策略")
print(f"   ✓ 理解内存管理和泄漏检测技术")
print(f"   ✓ 能够进行异步基准测试和性能对比")

### 事件循环优化技术 [⭐⭐进阶]
**知识点说明**：事件循环优化是提升异步系统性能的核心技术，对于LangChain的高并发处理和实时响应具有重要意义。

**学习要求**：
- 掌握事件循环性能调优方法
- 学会任务批处理和调度优化
- 理解连接池和资源管理技术
- 能够实现高效的事件循环策略

**案例要求**：
- 实现完整的事件循环优化方案
- 进行任务调度和批处理优化
- 应用优化技术解决实际问题
- 验证点：能独立优化异步事件循环性能

In [ ]:
print("\n🚀 事件循环优化技术:")
print("=" * 50)

# 2. 事件循环优化
print(f"📝 2. 事件循环优化:")

# 2.1 任务批处理优化
print(f"\n   🔍 2.1 任务批处理优化:")

class TaskBatchProcessor:
    """任务批处理器"""
    
    def __init__(self, batch_size: int = 10, batch_timeout: float = 0.1):
        self.batch_size = batch_size
        self.batch_timeout = batch_timeout
        
        # 任务队列
        self.task_queue = asyncio.Queue()
        self.batch_queue = asyncio.Queue()
        
        # 统计信息
        self.stats = {
            "total_tasks": 0,
            "batches_processed": 0,
            "average_batch_size": 0,
            "processing_time": 0.0
        }
        
        # 控制
        self.running = False
        self.processor_task: Optional[asyncio.Task] = None
    
    async def start(self):
        """启动批处理器"""
        if self.running:
            return
        
        self.running = True
        self.processor_task = asyncio.create_task(self._batch_processor_loop())
        
        print(f"      🚀 任务批处理器启动 (批大小: {self.batch_size}, 超时: {self.batch_timeout}s)")
    
    async def stop(self):
        """停止批处理器"""
        if not self.running:
            return
        
        self.running = False
        
        if self.processor_task:
            self.processor_task.cancel()
            try:
                await self.processor_task
            except asyncio.CancelledError:
                pass
        
        print(f"      🛑 任务批处理器停止")
    
    async def submit_task(self, task_func: Callable, *args, **kwargs):
        """提交任务"""
        task = {
            "func": task_func,
            "args": args,
            "kwargs": kwargs,
            "submitted_at": time.time()
        }
        
        await self.task_queue.put(task)
        self.stats["total_tasks"] += 1
    
    async def _batch_processor_loop(self):
        """批处理循环"""
        while self.running:
            try:
                # 收集批次
                batch = await self._collect_batch()
                
                if batch:
                    # 处理批次
                    await self._process_batch(batch)
                    
            except Exception as e:
                print(f"      ❌ 批处理循环错误: {e}")
                await asyncio.sleep(0.1)
    
    async def _collect_batch(self) -> List[Dict[str, Any]]:
        """收集任务批次"""
        batch = []
        deadline = time.time() + self.batch_timeout
        
        while len(batch) < self.batch_size and time.time() < deadline:
            try:
                timeout = max(0.001, deadline - time.time())
                task = await asyncio.wait_for(self.task_queue.get(), timeout=timeout)
                batch.append(task)
            except asyncio.TimeoutError:
                break
        
        return batch
    
    async def _process_batch(self, batch: List[Dict[str, Any]]):
        """处理任务批次"""
        start_time = time.time()
        
        # 并发执行批次中的所有任务
        tasks = []
        for task in batch:
            if asyncio.iscoroutinefunction(task["func"]):
                task_coro = task["func"](*task["args"], **task["kwargs"])
            else:
                # 同步函数在线程池中执行
                loop = asyncio.get_event_loop()
                task_coro = loop.run_in_executor(None, task["func"], *task["args"], **task["kwargs"])
            
            tasks.append(task_coro)
        
        # 等待所有任务完成
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # 更新统计
        processing_time = time.time() - start_time
        self.stats["batches_processed"] += 1
        self.stats["processing_time"] += processing_time
        
        # 计算平均批大小
        self.stats["average_batch_size"] = (
            self.stats["total_tasks"] / self.stats["batches_processed"]
        )
        
        # 统计结果
        successful = sum(1 for r in results if not isinstance(r, Exception))
        failed = len(results) - successful
        
        print(f"      📦 批次处理完成: {len(batch)} 个任务, "
              f"成功: {successful}, 失败: {failed}, "
              f"耗时: {processing_time:.3f}s")
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        stats = self.stats.copy()
        
        if stats["batches_processed"] > 0:
            stats["average_processing_time"] = (
                stats["processing_time"] / stats["batches_processed"]
            )
        else:
            stats["average_processing_time"] = 0
        
        stats["queue_size"] = self.task_queue.qsize()
        
        return stats

# 2.2 连接池优化
print(f"\n   🔗 2.2 连接池优化:")

class AsyncConnectionPool:
    """异步连接池"""
    
    def __init__(self, max_connections: int = 10, 
                 connection_timeout: float = 30.0,
                 idle_timeout: float = 300.0):
        self.max_connections = max_connections
        self.connection_timeout = connection_timeout
        self.idle_timeout = idle_timeout
        
        # 连接管理
        self.available_connections: asyncio.Queue = asyncio.Queue(maxsize=max_connections)
        self.active_connections: Dict[str, Dict[str, Any]] = {}
        self.connection_stats = {
            "created": 0,
            "reused": 0,
            "expired": 0,
            "failed": 0
        }
        
        # 清理任务
        self.cleanup_task: Optional[asyncio.Task] = None
        self.running = False
    
    async def start(self):
        """启动连接池"""
        if self.running:
            return
        
        self.running = True
        self.cleanup_task = asyncio.create_task(self._cleanup_loop())
        
        print(f"      🚀 异步连接池启动 (最大连接数: {self.max_connections})")
    
    async def stop(self):
        """停止连接池"""
        if not self.running:
            return
        
        self.running = False
        
        # 取消清理任务
        if self.cleanup_task:
            self.cleanup_task.cancel()
            try:
                await self.cleanup_task
            except asyncio.CancelledError:
                pass
        
        # 关闭所有连接
        while not self.available_connections.empty():
            try:
                conn = self.available_connections.get_nowait()
                await self._close_connection(conn)
            except asyncio.QueueEmpty:
                break
        
        print(f"      🛑 异步连接池停止")
    
    async def get_connection(self, connection_factory: Callable) -> str:
        """获取连接"""
        connection_id = str(uuid.uuid4())
        
        try:
            # 尝试从池中获取现有连接
            conn = await asyncio.wait_for(
                self.available_connections.get(),
                timeout=1.0
            )
            
            # 检查连接是否仍然有效
            if await self._is_connection_valid(conn):
                self.active_connections[connection_id] = {
                    "connection": conn,
                    "created_at": time.time(),
                    "last_used": time.time()
                }
                self.connection_stats["reused"] += 1
                return connection_id
            else:
                # 连接无效，关闭并创建新的
                await self._close_connection(conn)
                self.connection_stats["expired"] += 1
        
        except asyncio.TimeoutError:
            # 池中没有可用连接，创建新的
            pass
        
        # 创建新连接
        try:
            new_conn = await asyncio.wait_for(
                connection_factory(),
                timeout=self.connection_timeout
            )
            
            self.active_connections[connection_id] = {
                "connection": new_conn,
                "created_at": time.time(),
                "last_used": time.time()
            }
            
            self.connection_stats["created"] += 1
            return connection_id
            
        except Exception as e:
            self.connection_stats["failed"] += 1
            raise Exception(f"创建连接失败: {e}")
    
    async def release_connection(self, connection_id: str):
        """释放连接"""
        if connection_id not in self.active_connections:
            return
        
        conn_info = self.active_connections.pop(connection_id)
        conn_info["last_used"] = time.time()
        
        try:
            # 将连接放回池中
            await self.available_connections.put(conn_info)
        except asyncio.QueueFull:
            # 池满了，关闭连接
            await self._close_connection(conn_info["connection"])
    
    async def _cleanup_loop(self):
        """清理循环"""
        while self.running:
            try:
                await self._cleanup_expired_connections()
                await asyncio.sleep(60)  # 每分钟清理一次
            except Exception as e:
                print(f"      ❌ 连接池清理错误: {e}")
                await asyncio.sleep(10)
    
    async def _cleanup_expired_connections(self):
        """清理过期连接"""
        current_time = time.time()
        expired_connections = []
        
        # 检查活跃连接
        for conn_id, conn_info in list(self.active_connections.items()):
            if current_time - conn_info["last_used"] > self.idle_timeout:
                expired_connections.append(conn_id)
        
        # 清理过期连接
        for conn_id in expired_connections:
            conn_info = self.active_connections.pop(conn_id, None)
            if conn_info:
                await self._close_connection(conn_info["connection"])
                self.connection_stats["expired"] += 1
    
    async def _is_connection_valid(self, conn) -> bool:
        """检查连接是否有效"""
        # 简化实现，实际应该根据具体连接类型检查
        return True
    
    async def _close_connection(self, conn):
        """关闭连接"""
        # 简化实现，实际应该根据具体连接类型关闭
        pass
    
    def get_stats(self) -> Dict[str, Any]:
        """获取连接池统计"""
        return {
            **self.connection_stats,
            "available_connections": self.available_connections.qsize(),
            "active_connections": len(self.active_connections),
            "pool_utilization": (
                (self.max_connections - self.available_connections.qsize()) / self.max_connections * 100
            )
        }

# 2.3 信号量优化
print(f"\n   🚦 2.3 信号量优化:")

class AdaptiveSemaphore:
    """自适应信号量"""
    
    def __init__(self, initial_value: int = 10, 
                 max_value: int = 100, 
                 min_value: int = 1):
        self.max_value = max_value
        self.min_value = min_value
        
        # 信号量
        self.semaphore = asyncio.Semaphore(initial_value)
        self.current_value = initial_value
        
        # 性能监控
        self.wait_times: deque = deque(maxlen=100)
        self.utilization_history: deque = deque(maxlen=100)
        
        # 调整参数
        self.adjustment_interval = 30.0
        self.high_wait_threshold = 0.1  # 100ms
        self.low_utilization_threshold = 0.3  # 30%
        
        # 控制
        self.adjustment_task: Optional[asyncio.Task] = None
        self.running = False
    
    async def start(self):
        """启动自适应调整"""
        if self.running:
            return
        
        self.running = True
        self.adjustment_task = asyncio.create_task(self._adjustment_loop())
        
        print(f"      🚀 自适应信号量启动 (初始值: {self.current_value})")
    
    async def stop(self):
        """停止自适应调整"""
        if not self.running:
            return
        
        self.running = False
        
        if self.adjustment_task:
            self.adjustment_task.cancel()
            try:
                await self.adjustment_task
            except asyncio.CancelledError:
                pass
        
        print(f"      🛑 自适应信号量停止")
    
    async def acquire(self) -> int:
        """获取信号量"""
        start_time = time.time()
        
        await self.semaphore.acquire()
        
        wait_time = time.time() - start_time
        self.wait_times.append(wait_time)
        
        # 更新利用率
        utilization = 1 - (self.semaphore._value / self.current_value)
        self.utilization_history.append(utilization)
        
        return self.semaphore._value
    
    def release(self):
        """释放信号量"""
        self.semaphore.release()
    
    async def _adjustment_loop(self):
        """调整循环"""
        while self.running:
            try:
                await self._adjust_semaphore()
                await asyncio.sleep(self.adjustment_interval)
            except Exception as e:
                print(f"      ❌ 信号量调整错误: {e}")
                await asyncio.sleep(10)
    
    async def _adjust_semaphore(self):
        """调整信号量值"""
        if not self.wait_times or not self.utilization_history:
            return
        
        # 计算平均等待时间和利用率
        avg_wait_time = sum(self.wait_times) / len(self.wait_times)
        avg_utilization = sum(self.utilization_history) / len(self.utilization_history)
        
        old_value = self.current_value
        
        # 调整逻辑
        if avg_wait_time > self.high_wait_threshold and self.current_value < self.max_value:
            # 等待时间过长，增加信号量值
            self.current_value = min(self.max_value, self.current_value + 2)
            await self._update_semaphore_value()
            print(f"      ⬆️ 信号量值增加: {old_value} -> {self.current_value} (等待时间: {avg_wait_time:.3f}s)")
            
        elif avg_utilization < self.low_utilization_threshold and self.current_value > self.min_value:
            # 利用率过低，减少信号量值
            self.current_value = max(self.min_value, self.current_value - 1)
            await self._update_semaphore_value()
            print(f"      ⬇️ 信号量值减少: {old_value} -> {self.current_value} (利用率: {avg_utilization:.1%})")
    
    async def _update_semaphore_value(self):
        """更新信号量值"""
        # 计算需要释放或获取的数量
        diff = self.current_value - self.semaphore._value
        
        if diff > 0:
            # 需要增加信号量值
            for _ in range(diff):
                self.semaphore.release()
        elif diff < 0:
            # 需要减少信号量值
            for _ in range(abs(diff)):
                if self.semaphore._value > 0:
                    await self.semaphore.acquire()
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        if not self.wait_times or not self.utilization_history:
            return {
                "current_value": self.current_value,
                "available_permits": self.semaphore._value,
                "average_wait_time": 0,
                "average_utilization": 0
            }
        
        return {
            "current_value": self.current_value,
            "available_permits": self.semaphore._value,
            "average_wait_time": sum(self.wait_times) / len(self.wait_times),
            "average_utilization": sum(self.utilization_history) / len(self.utilization_history),
            "max_wait_time": max(self.wait_times),
            "max_utilization": max(self.utilization_history)
        }

print(f"   ✅ 事件循环优化技术完成")
print(f"      - TaskBatchProcessor: 任务批处理优化")
print(f"      - AsyncConnectionPool: 连接池管理")
print(f"      - AdaptiveSemaphore: 自适应信号量")

print(f"\n✅ 事件循环优化技术完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握事件循环性能调优方法")
print(f"   ✓ 学会任务批处理和调度优化")
print(f"   ✓ 理解连接池和资源管理技术")
print(f"   ✓ 能够实现高效的事件循环策略")

### LangChain性能优化实践 [⭐⭐进阶]
**知识点说明**：LangChain性能优化实践是掌握AI系统性能提升的关键，对于LangChain的大规模部署和用户体验具有重要意义。

**学习要求**：
- 掌握LangChain推理性能优化
- 学会提示词缓存和批处理技术
- 理解流式响应和并发处理
- 能够构建高性能LangChain应用

**案例要求**：
- 实现完整的LangChain性能优化方案
- 进行AI推理性能对比测试
- 应用优化技术解决实际问题
- 验证点：能独立优化LangChain应用性能

In [ ]:
# 3. 内存泄漏检测器
print(f"\n   🔍 3. 内存泄漏检测器:")

@dataclass
class MemorySnapshot:
    """内存快照"""
    timestamp: float = field(default_factory=time.time)
    current_memory: int = 0  # MB
    peak_memory: int = 0     # MB
    object_count: int = 0
    gc_stats: Dict[str, int] = field(default_factory=dict)

class AsyncMemoryLeakDetector:
    """异步内存泄漏检测器"""
    
    def __init__(self, snapshot_interval: float = 10.0, memory_growth_threshold: float = 50.0):
        self.snapshot_interval = snapshot_interval
        self.memory_growth_threshold = memory_growth_threshold
        self.snapshots: List[MemorySnapshot] = []
        self.object_refs: List[weakref.ref] = []
        self.monitoring = False
        self.monitor_task: Optional[asyncio.Task] = None
        
        # 启动内存跟踪
        tracemalloc.start()
    
    async def start_monitoring(self):
        """开始监控"""
        if self.monitoring:
            return
        
        self.monitoring = True
        self.monitor_task = asyncio.create_task(self._monitor_loop())
        
        print(f"      🚀 内存泄漏检测启动 (间隔: {self.snapshot_interval}s)")
    
    async def stop_monitoring(self):
        """停止监控"""
        if not self.monitoring:
            return
        
        self.monitoring = False
        
        if self.monitor_task:
            self.monitor_task.cancel()
            try:
                await self.monitor_task
            except asyncio.CancelledError:
                pass
        
        print(f"      🛑 内存泄漏检测停止")
    
    def track_object(self, obj: Any) -> str:
        """跟踪对象"""
        obj_id = str(uuid.uuid4())
        self.object_refs.append(weakref.ref(obj, lambda ref: self._on_object_deleted(obj_id)))
        return obj_id
    
    def _on_object_deleted(self, obj_id: str):
        """对象被删除时的回调"""
        print(f"      🗑️ 对象已清理: {obj_id[:8]}")
    
    async def _monitor_loop(self):
        """监控循环"""
        while self.monitoring:
            try:
                # 创建内存快照
                snapshot = await self._create_memory_snapshot()
                self.snapshots.append(snapshot)
                
                # 检测内存泄漏
                await self._detect_memory_leaks()
                
                # 限制快照数量
                if len(self.snapshots) > 100:
                    self.snapshots = self.snapshots[-100:]
                
                await asyncio.sleep(self.snapshot_interval)
                
            except Exception as e:
                print(f"      ❌ 内存监控循环错误: {e}")
                await asyncio.sleep(5)
    
    async def _create_memory_snapshot(self) -> MemorySnapshot:
        """创建内存快照"""
        # 获取当前内存使用情况
        current, peak = tracemalloc.get_traced_memory()
        current_mb = current / 1024 / 1024
        peak_mb = peak / 1024 / 1024
        
        # 获取对象计数
        object_count = len(self.object_refs)
        
        # 获取GC统计
        gc_stats = {
            "counts": gc.get_count(),
            "threshold": gc.get_threshold()
        }
        
        return MemorySnapshot(
            current_memory=current_mb,
            peak_memory=peak_mb,
            object_count=object_count,
            gc_stats=gc_stats
        )
    
    async def _detect_memory_leaks(self):
        """检测内存泄漏"""
        if len(self.snapshots) < 2:
            return
        
        # 比较最近的快照
        current = self.snapshots[-1]
        previous = self.snapshots[-2]
        
        # 检查内存增长
        memory_growth = current.current_memory - previous.current_memory
        if memory_growth > self.memory_growth_threshold:
            print(f"      ⚠️ 检测到内存增长: {memory_growth:.1f}MB")
        
        # 检查对象增长
        object_growth = current.object_count - previous.object_count
        if object_growth > 10:
            print(f"      ⚠️ 检测到对象增长: {object_growth} 个")
    
    def get_memory_stats(self) -> Dict[str, Any]:
        """获取内存统计"""
        if not self.snapshots:
            return {}
        
        latest = self.snapshots[-1]
        
        return {
            "current_memory_mb": latest.current_memory,
            "peak_memory_mb": latest.peak_memory,
            "tracked_objects": latest.object_count,
            "gc_stats": latest.gc_stats,
            "snapshots_count": len(self.snapshots)
        }

# 4. 异步调试分析器
print(f"\n   🐛 4. 异步调试分析器:")

class AsyncDebugProfiler:
    """异步调试分析器"""
    
    def __init__(self):
        self.profiler: Optional[cProfile.Profile] = None
        self.asyncio_debug_enabled = False
        self.original_slow_callback_duration = None
    
    def enable_asyncio_debug(self):
        """启用asyncio调试模式"""
        if not self.asyncio_debug_enabled:
            # 启用调试模式
            loop = asyncio.get_event_loop()
            loop.set_debug(True)
            
            # 设置慢回调阈值
            self.original_slow_callback_duration = loop.slow_callback_duration
            loop.slow_callback_duration = 0.1  # 100ms
            
            self.asyncio_debug_enabled = True
            print(f"      🐛 asyncio调试模式已启用")
    
    def disable_asyncio_debug(self):
        """禁用asyncio调试模式"""
        if self.asyncio_debug_enabled:
            loop = asyncio.get_event_loop()
            loop.set_debug(False)
            
            if self.original_slow_callback_duration:
                loop.slow_callback_duration = self.original_slow_callback_duration
            
            self.asyncio_debug_enabled = False
            print(f"      🐛 asyncio调试模式已禁用")
    
    def start_profiling(self):
        """开始性能分析"""
        self.profiler = cProfile.Profile()
        self.profiler.enable()
        
        print(f"      📈 性能分析已开始")
    
    def stop_profiling(self) -> Dict[str, Any]:
        """停止性能分析并获取结果"""
        if not self.profiler:
            return {}
        
        self.profiler.disable()
        
        # 获取统计信息
        stream = io.StringIO()
        stats = pstats.Stats(self.profiler, stream=stream)
        stats.sort_stats('cumulative')
        stats.print_stats(20)  # 打印前20个函数
        
        # 解析结果
        result_text = stream.getvalue()
        
        return {
            "profiler_output": result_text,
            "total_calls": stats.total_calls,
        }
    
    @asynccontextmanager
    async def profile_context(self, enable_asyncio_debug: bool = True):
        """性能分析上下文管理器"""
        if enable_asyncio_debug:
            self.enable_asyncio_debug()
        
        self.start_profiling()
        
        try:
            yield self
        finally:
            results = self.stop_profiling()
            
            if enable_asyncio_debug:
                self.disable_asyncio_debug()
            
            print(f"      📊 性能分析结果:")
            print(f"         总调用次数: {results.get('total_calls', 0)}")
            
            # 显示关键性能数据
            profiler_output = results.get('profiler_output', '')
            if profiler_output:
                lines = profiler_output.split('\n')
                for line in lines[:10]:  # 显示前10行
                    if line.strip():
                        print(f"         {line}")

print(f"   ✅ 异步调试和内存检测完成")
print(f"      - AsyncMemoryLeakDetector: 内存泄漏检测和对象跟踪")
print(f"      - AsyncDebugProfiler: asyncio调试和性能分析")
print(f"      - 完整的内存监控和性能分析工具")

print(f"\n✅ 异步调试和内存检测完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步程序内存泄漏检测技术")
print(f"   ✓ 学会使用tracemalloc和weakref进行内存分析")
print(f"   ✓ 熟练运用asyncio调试模式定位问题")
print(f"   ✓ 能够使用cProfile分析异步程序性能")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**6.8 异步性能优化 [⭐⭐进阶]**
- ✅ 掌握异步性能分析和监控工具
- ✅ 学会事件循环优化和任务调度策略
- ✅ 理解内存管理和泄漏检测技术
- ✅ 能够进行异步基准测试和性能对比
- ✅ 掌握事件循环性能调优方法
- ✅ 学会任务批处理和调度优化
- ✅ 理解连接池和资源管理技术
- ✅ 能够实现高效的事件循环策略
- ✅ 掌握LangChain推理性能优化
- ✅ 学会提示词缓存和批处理技术
- ✅ 理解流式响应和并发处理
- ✅ 能够构建高性能LangChain应用
- ✅ 能独立进行异步系统性能调优
- ✅ 能独立优化异步事件循环性能
- ✅ 能独立优化LangChain应用性能

### 🎯 与LangChain学习的关联

**异步性能优化重要性**：
- 性能优化提升LangChain的响应速度
- 监控工具确保LangChain的稳定性
- 基准测试验证LangChain的优化效果
- 内存管理保障LangChain的大规模部署
- 批处理优化增强LangChain的吞吐量

**实际应用场景**：
- LangChain的大规模模型推理优化
- LangChain的实时对话系统性能调优
- LangChain的文档分析流水线优化
- LangChain的多模态AI应用加速
- LangChain的企业级服务部署优化

### 📚 进阶学习建议

1. **练习建议**：
   - 深入练习异步性能分析和调优
   - 掌握复杂异步系统的优化策略
   - 学习大规模异步系统的架构设计

2. **扩展学习**：
   - 了解分布式系统性能优化
   - 学习云原生异步应用优化
   - 探索AI模型推理加速技术

3. **实际应用**：
   - 构建高性能LangChain推理平台
   - 开发企业级异步监控系统
   - 实现智能化的性能调优系统

### 🔧 常见错误与注意事项

1. **过度优化问题**：
   ```python
   # 错误：过度优化导致代码复杂
   async def over_optimized_function():
       # 过多的手动优化，难以维护
       if condition1:
           if condition2:
               if condition3:
                   # 嵌套过深的优化逻辑
                   await optimized_call()
   
   # 正确：保持代码简洁，测量后优化
   async def clean_function():
       # 清晰的逻辑，必要时再优化
       result = await standard_call()
       return result
   ```

2. **盲目批处理**：
   ```python
   # 错误：不考虑实际场景的批处理
   async def bad_batch_processor():
       while True:
           batch = await collect_batch(size=100)  # 固定大批次
           await process_batch(batch)  # 可能导致延迟过高
   
   # 正确：根据负载动态调整批处理
   async def smart_batch_processor():
       while True:
           # 根据队列大小和延迟要求调整批次
           batch_size = calculate_optimal_batch_size()
           batch = await collect_batch(size=batch_size, timeout=dynamic_timeout)
           await process_batch(batch)
   ```

3. **缓存滥用**：
   ```python
   # 错误：缓存所有内容，内存爆炸
   class BadCache:
       def __init__(self):
           self.cache = {}  # 无限制缓存
       
       def get(self, key):
           return self.cache.get(key)
       
       def put(self, key, value):
           self.cache[key] = value  # 永不清理
   
   # 正确：实现智能缓存策略
   class SmartCache:
       def __init__(self, max_size=1000, ttl=3600):
           self.cache = {}
           self.max_size = max_size
           self.ttl = ttl
       
       def put(self, key, value):
           if len(self.cache) >= self.max_size:
               self.evict_lru()
           self.cache[key] = {
               'value': value,
               'created_at': time.time()
           }
   ```

4. **监控缺失**：
   ```python
   # 错误：没有性能监控
   async def unmonitored_service():
       while True:
           await process_requests()  # 不知道性能如何
   
   # 正确：实现全面的性能监控
   async def monitored_service():
       monitor = PerformanceMonitor()
       await monitor.start()
       
       try:
           while True:
               start_time = time.time()
               await process_requests()
               
               # 记录性能指标
               monitor.record_processing_time(time.time() - start_time)
               
               # 检查性能告警
               if monitor.get_average_latency() > threshold:
                   await trigger_performance_alert()
       
       finally:
           await monitor.stop()
   ```

5. **资源泄漏**：
   ```python
   # 错误：资源没有正确释放
   async def resource_leak_example():
       connection = await create_connection()
       await connection.execute(query)
       # 忘记关闭连接，导致资源泄漏
   
   # 正确：使用上下文管理器确保资源释放
   async def resource_safe_example():
       async with connection_pool.get_connection() as conn:
           await conn.execute(query)
       # 连接自动释放
   ```

6. **同步阻塞异步**：
   ```python
   # 错误：在异步代码中使用同步阻塞操作
   async def blocking_async_function():
       data = requests.get(url)  # 阻塞整个事件循环
       await process_data(data)
   
   # 正确：使用异步替代方案
   async def non_blocking_async_function():
       async with aiohttp.ClientSession() as session:
           async with session.get(url) as response:
               data = await response.json()
       await process_data(data)
   ```

### 🌐 性能优化最佳实践

**监控驱动优化**：
- 建立全面的性能监控体系
- 基于数据驱动的优化决策
- 持续监控和调整优化策略
- 建立性能基线和回归测试

**渐进式优化**：
- 先解决明显的性能瓶颈
- 逐步优化，避免过度设计
- 每次优化都要进行效果验证
- 保持代码的可读性和可维护性

**LangChain特定优化**：
- 针对不同模型的优化策略
- 智能提示词缓存和去重
- 批量推理和流水线处理
- 流式响应和增量更新

---

**🎉 恭喜完成异步性能优化学习！**

你已经全面掌握了异步性能优化的核心技术，能够系统性地进行性能分析、事件循环优化、LangChain性能调优等，为LangChain智能应用提供了强大的性能保障。

## 🚀 下一步学习预告

**第六节：异步编程 - 进行中** 🔄
- 6.1 异步编程基础 ✅
- 6.2 协程与事件循环 ✅
- 6.3 异步I/O操作 ✅
- 6.4 异步网络编程 ✅
- 6.5 异步Web框架 ✅
- 6.6 并发编程模式 ✅
- 6.7 异步任务队列 ✅
- 6.8 异步性能优化 ✅
- 6.9 异步最佳实践 ⏳

**继续第六节：异步编程**
- 下一个：6.9 异步最佳实践
- 深入学习异步编程的最佳实践和设计模式
- 掌握异步系统的架构设计和部署策略

**后续章节预告**：
- Web开发技术
- 项目工程实践

继续加油，异步性能优化已经掌握！准备学习异步最佳实践！